In [1]:
import nbimporter
import copy

import model_house
import get_u_and_eta_from_u_a_and_eta_a as get_u_and_eta

Importing Jupyter notebook from model_house.ipynb
Importing Jupyter notebook from get_u_and_eta_from_u_a_and_eta_a.ipynb
Importing Jupyter notebook from nu.ipynb
Importing Jupyter notebook from f.ipynb
Importing Jupyter notebook from H.ipynb


# 外皮情報LV0からLV1へのコンバート

## I. 統合

面積・長さは床面積90.0㎡の簡易モデル住宅モデルより、延床面積に応じて決定する。  
窓のf値は、簡易モデル住宅モデルの窓方位より決定する。  
部位のU値・η値は、UA値・η値より決定する。

In [2]:
def convert(d):
    
    # 簡易モデル住宅をもとに面積を取得
    d_for_u_and_eta, f_heating, f_cooling = model_house.convert(d, 'not_output_spec')
    d_for_u_and_eta['envelope']['index'] = d['envelope']['index']
    
    # U値、η値の取得 
    eta, eta_heating, eta_cooling, part_U = get_u_and_eta.calc_adjustment_factor( d_for_u_and_eta )
    
    # 出力
    return {
        'common' : copy.deepcopy(d['common']),  
        'envelope' : {
            'simple_method' : {
                'insulation_type'          : 'floor',
                'insulation_type_bathroom' : 'inside',
                'u_value_roof'             : part_U['roof'],
                'u_value_wall'             : part_U['wall'],
                'u_value_floor_other'      : part_U['floor'],
                'u_value_door'             : part_U['window'],
                'u_value_window'           : part_U['window'],
                'eta_d_value_window_c'     : eta_cooling,       
                'eta_d_value_window_h'     : eta_heating, 
                'is_f_value_input'         : True, 
                'f_value_c'                : f_cooling,
                'f_value_h'                : f_heating,
                'psi_value_earthfloor_perimeter_entrance' : part_U['earthfloor_perimeter']
            }
        }
    }

### Example

In [3]:
d = {
    'common' : {
        'region' : 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 90.0
    },
    'envelope' : {
        'input_method' : 'index',
        'index' :{
            'UA' : 0.87,
            'etaAH': 2.8,
            'etaAC': 1.4
        }
    }
}

convert(d)

U値{'roof': 0.20174395772715983, 'ceiling': 0.20174395772715983, 'wall': 0.445517906647478, 'floor': 0.4034879154543204, 'window': 3.9087891809637223, 'door': 3.9087891809637223, 'earthfloor_perimeter': 0.6388558661360073},開口部η0.22494713216779566,開口部ηH0.3679210611053846,開口部ηC0.22494713216779566
UA計算値0.87,ηAH計算値2.8000000000000003,ηAC計算値1.4


{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 30.0,
  'total_floor_area': 90.0},
 'envelope': {'simple_method': {'insulation_type': 'floor',
   'insulation_type_bathroom': 'inside',
   'u_value_roof': 0.20174395772715983,
   'u_value_wall': 0.445517906647478,
   'u_value_floor_other': 0.4034879154543204,
   'u_value_door': 3.9087891809637223,
   'u_value_window': 3.9087891809637223,
   'eta_d_value_window_c': 0.22494713216779566,
   'eta_d_value_window_h': 0.3679210611053846,
   'is_f_value_input': True,
   'f_value_c': 0.664,
   'f_value_h': 0.655,
   'psi_value_earthfloor_perimeter_entrance': 0.6388558661360073}}}